In [14]:
from __future__ import print_function, division
import h5py
import os
from preprocessor import _patchSampler, _patchExtractor
import re
import SimpleITK as sitk
import numpy as np
from pylab import *

#maxratio: to balance the images (0 for test set)
def n_patchGenerator(listFile, patchSize, masktype='bmask',outputFile='', maxratio=2, channels=6):
    assert(not patchSize % 2 == 0)
    nChannels = channels #'T1','T2','FLAIR'
    PS = int(patchSize/2)
#     print(PS)
#     PS = int(PS)
#     print(PS.dtype.kind)
    imageFile = listFile['S1_FLAIR']
    GTFile = listFile['GT']

    GT = sitk.ReadImage(listFile['GT'])
    GT = sitk.GetArrayFromImage(GT)
    print('GT:'+str(np.count_nonzero(GT)))
    brainmask = sitk.ReadImage(listFile['brainmask'])
    brainmask = sitk.GetArrayFromImage(brainmask)
    print('brainmask:'+str(np.count_nonzero(brainmask)))
    mask = brainmask
#     mask = _patchExtractor(imageFile, GTFile, masktype)
#     print(len(mask[0][0]), len(mask[0]), len(mask)) 715*715*715
    # balance the classes if wanted
    # balance the classes if wanted
    if maxratio>0:
        labs = unique(GT[mask>0])
        cards = dict.fromkeys(labs,0)
        for lab in labs:
            cards[lab]=sum(GT[mask>0]==lab)
        maxcard = min(cards.values())*maxratio
        for lab in labs:
            # delete the exceeding points randomly
            delcard = max(0,cards[lab]-maxcard)
            if delcard>0:
                inds = np.arange(cards[lab])
                shuffle(inds)
                inds = inds[:delcard]
                _mask = (GT==lab) & (mask>0)
                INDS = np.flatnonzero(_mask)
                mask[np.unravel_index(INDS[inds],_mask.shape)]=0


    [IZZ,IRR,ICC] = np.where(mask>0)
    npatch = size(IZZ)
#     print(size(IZZ), size(IRR), size(ICC))
    # if needed, record directly to the output file
    if not outputFile=='':
        # write into a single hdf5 file
        import h5py
        with h5py.File(outputFile,'w') as patchFile:
            _datashape = [npatch,nChannels,patchSize,patchSize]
            _patches = patchFile.create_dataset('data', shape=_datashape,dtype='float32',compression="gzip")
            _labelshape = [npatch]
            _labels = patchFile.create_dataset('labels', shape=_labelshape,dtype='uint8',compression="gzip")
            
            bSize = min(npatch,250000) # split the original volume into patches we can have in RAM 
                            
#             imageChannelCount = 0
#             for smod in {'S1_T1','S1_T2','S1_FLAIR'}:
#                 timepoints = [smod,'S2_%s'%(smod[3:])]
#                 ndas = []
#                 for name in timepoints:
#                     inputImage = sitk.ReadImage(listFile[name])
#                     nda = np.float32(sitk.GetArrayFromImage(inputImage))
#                     # compute z-scores ('standardization')
#                     ind = np.where(mask>0)
#                     nda[ind] = (nda[ind] - np.mean(nda[ind]))/ (np.std(nda[ind])+np.finfo(float).eps)
#                     nda = np.lib.pad(nda,((PS,PS)), 'constant',constant_values=(0,0))
#                     ndas.append(nda)
#                 dist = ndas[1] - ndas[0] #S2 - S1
#                 distdeltaT = dist * deltaT
#                 for ipatch in range(npatch):
#                     z = IZZ[ipatch]+PS; i = IRR[ipatch]+PS; j = ICC[ipatch]+PS
#                     patches[ipatch,imageChannelCount,:,:] = ndas[0][z,i-PS:i+PS+1,j-PS:j+PS+1]
#                     patches[ipatch,(imageChannelCount+1),:,:] = dist[z,i-PS:i+PS+1,j-PS:j+PS+1]
#                     patches[ipatch,(imageChannelCount+1),:,:] = distdeltaT[z,i-PS:i+PS+1,j-PS:j+PS+1]
#                     ipatch +=1
#                 imageChannelCount += 3
                
            imageChannelCount = 0
            NDA = []
            for smod in ['delta_FLAIR']:# TODO: smod-s for differences etc -- to be stored as volumes with "key" names, eg "deltaT1" 
                #'S1_T1','S1_T2','S1_FLAIR', 'delta_T1', 'delta_T2', 'delta_FLAIR'
                inputImage = sitk.ReadImage(listFile[smod])
                nda = np.float32(sitk.GetArrayFromImage(inputImage))
#                 # compute z-scores ('standardization')
#                 ind = np.where(nda>0)
#                 nda[ind] = (nda[ind] - np.mean(nda[ind]))/(np.std(nda[ind])+np.finfo(float).eps)
                nda = np.lib.pad(nda,((PS,PS),(PS,PS),(PS,PS)), 'constant',constant_values=(0,0))
                NDA.append(nda)
                imageChannelCount += 1
            
            print('npatches:'+str(npatch))
            for ibatch in range(0,npatch,bSize):
                _shape = _datashape
                _shape[0] = min(bSize,npatch-ibatch)
                patches = np.zeros(_shape)
                print(ibatch/float(bSize),_shape) 
                for imageChannelCount in range(nChannels):
                    print('imageChannelCount:'+str(imageChannelCount))
                    nda = NDA[imageChannelCount]
                    for ipatch in range(_shape[0]):
                        z = IZZ[ibatch+ipatch]+PS; i = IRR[ibatch+ipatch]+PS; j = ICC[ibatch+ipatch]+PS
                        #if patchtype=='axial':
                        patches[ipatch,imageChannelCount,:,:] = nda[z,i-PS:i+PS+1,j-PS:j+PS+1]
                        #elif patchtype=='3D':
                        #    patches[ipatch,imageChannelCount,:,:,:] = nda[z-PS:z+PS+1,i-PS:i+PS+1,j-PS:j+PS+1]

                _patches[ibatch:min(ibatch+bSize,npatch),...] = patches

            # assuming a single label per patch, and that they fit into memory
            GT = sitk.ReadImage(listFile['GT'])
            GT = sitk.GetArrayFromImage(GT)
            print( unique(GT[mask>0].flatten()))
            _labels[:] = GT[mask>0]
            print('done.')
            return
            
#         import h5py
#         with h5py.File(outputFile,'w') as patchFile:
#             _datashape = [npatch,nChannels,patchSize,patchSize]
#             patches = patchFile.create_dataset('data', shape=_datashape,dtype='float32')
#             _labelshape = [npatch]
#             labels = patchFile.create_dataset('labels', shape=_labelshape,dtype='uint8')
            
#             cumpatch = 0; ifile = 0            
#             bSize = min(npatch,10000) 
#             _npatch = npatch-cumpatch
#             print(_npatch)
    else:
        print(npatch*nChannels*patchSize*patchSize*4)
        patches = np.zeros(shape=(npatch,nChannels,patchSize, patchSize))
        NDA = []
        for smod in {'delta_FLAIR'}:
            inputImage = sitk.ReadImage(listFile[smod])
            nda = np.float32(sitk.GetArrayFromImage(inputImage))
            nda = np.lib.pad(nda,((PS,PS),(PS,PS),(PS,PS)), 'constant',constant_values=(0,0))
            NDA.append(nda)
            
        imageChannelCount = 0
        for imageChannelCount in range(nChannels):
                print('imageChannelCount:'+str(imageChannelCount))
                nda = NDA[imageChannelCount]
                for ipatch in range(npatch):
                    z = IZZ[ipatch]+PS; i = IRR[ipatch]+PS; j = ICC[ipatch]+PS
                    #if patchtype=='axial':
                    patches[ipatch,imageChannelCount,:,:] = nda[z,i-PS:i+PS+1,j-PS:j+PS+1]
                    #elif patchtype=='3D':
                    #    patches[ipatch,imageChannelCount,:,:,:] = nda[z-PS:z+PS+1,i-PS:i+PS+1,j-PS:j+PS+1]

        labels = GT[mask>0]
        

        return patches, labels, mask

In [16]:
#create the patches

channels = 6
# deltaT = 30
dataDir = '../../MSpatientdata'
newDataDir = '../../MSpatientdata-normalized/zscore'
# subjectID = 1
subjectIDs = [1,2,4,5,6,7,8,10,12,14,15,16,17,18,19]
trainingIDs = [1,2,4,5,6,7,8,12,14,16,17,19]
# trainingIDs = []
# testIDs = [10, 15, 18]
testIDs=[]
images = []
lbls = []
pSizes = [29] #patch sizes
listFile = {}
filename = {}
filename['S1_T1'] = 'study1_T1'
filename['S1_T2'] = 'study1_T2'
filename['delta_T1'] = 'delta_T1'
filename['delta_T2'] = 'delta_T2'
filename['S1_FLAIR'] = 'study1_FLAIR' 
filename['delta_FLAIR'] = 'delta_FLAIR'
filename['GT'] = 'gt3'
filename['brainmask'] = 'brainmask'

for pSize in pSizes:
    for subjectID in subjectIDs:
        
        for smod in {'S1_T1','S1_T2','delta_T1','delta_T2','S1_FLAIR', 'delta_FLAIR'}:
            listFile[smod] = '%s/patient%d/patient%d_%s.nii.gz'%(newDataDir, subjectID, subjectID, filename[smod])
        for smod in {'GT','brainmask'}:
            listFile[smod] = '%s/patient%d/patient%d_%s.nii.gz'%(dataDir, subjectID, subjectID, filename[smod])

        # generate training patches:
        #if subjectID in trainingIDs:
        mode = 'training'
        outputDir = 'PATCHES'
        outputFile = '%s/patient%s_%sPatches%d.hdf5'%(outputDir, subjectID, mode, pSize)
        n_patchGenerator(listFile, pSize, masktype='bmask',outputFile=outputFile, maxratio=2, channels=1)

        continue # create only new training
    
        # generate test patches (not balanced):
        #if subjectID in testIDs:
        mode = 'test'
        outputDir = 'PATCHES'
        outputFile = '%s/patient%s_%sPatches%d.hdf5'%(outputDir, subjectID, mode, pSize)
        print('creating '+outputFile)
        n_patchGenerator(listFile, pSize, masktype='bmask',outputFile=outputFile, maxratio=0, channels = 1)
#         break
#     break

        #         split = 10
#         i = 0
#         while i < 5: 
#             outputFile = '%s/patient%s_%sPatches%d_%d.hdf5'%(outputDir, subjectID, mode, pSize, i+1)
#             print('creating '+outputFile)
#             with h5py.File(outputFile,'w') as patchFile:
#                 arraySize = int(len(testpatches)/split)
#                 _patches = patchFile.create_dataset('data', data = testpatches[(arraySize*i):(arraySize*(i+1))],dtype='float32',compression="gzip")
#                 _labels = patchFile.create_dataset('labels', data = testlabels[(arraySize*i):(arraySize*(i+1))],dtype='uint8',compression="gzip")
#             i += 2

GT:2264
brainmask:844248
npatches:1520
0.0 [1520, 1, 29, 29]
imageChannelCount:0
[0 1 2]
done.
GT:416
brainmask:597017
npatches:730
0.0 [730, 1, 29, 29]
imageChannelCount:0
[0 1 2]
done.
GT:516
brainmask:594345
npatches:1434
0.0 [1434, 1, 29, 29]
imageChannelCount:0
[0 1]
done.
GT:1620
brainmask:525540
npatches:2625
0.0 [2625, 1, 29, 29]
imageChannelCount:0
[0 1 2]
done.
GT:1667
brainmask:561019
npatches:2359
0.0 [2359, 1, 29, 29]
imageChannelCount:0


KeyboardInterrupt: 

In [3]:
print(listFile)

{'S1_T2': '../../MSpatientdata-normalized/zscore/patient1/patient1_study1_T2.nii.gz', 'delta_FLAIR': '../../MSpatientdata-normalized/zscore/patient1/patient1_delta_FLAIR.nii.gz', 'S1_FLAIR': '../../MSpatientdata-normalized/zscore/patient1/patient1_study1_FLAIR.nii.gz', 'GT': '../../MSpatientdata/patient1/patient1_gt3.nii.gz', 'delta_T1': '../../MSpatientdata-normalized/zscore/patient1/patient1_delta_T1.nii.gz', 'S1_T1': '../../MSpatientdata-normalized/zscore/patient1/patient1_study1_T1.nii.gz', 'delta_T2': '../../MSpatientdata-normalized/zscore/patient1/patient1_delta_T2.nii.gz', 'brainmask': '../../MSpatientdata/patient1/patient1_brainmask.nii.gz'}


In [ ]:
# visualize the patches

In [ ]:
# check the patch validity: reconstuct the gt volume from the labels in hdf5

In [ ]:
# check the patch validity: reconstuct an image volume from the channel (central point) in hdf5